In [1]:
import zipfile

import numpy as np
import pandas as pd
import tensorflow as tf

from datasets import Dataset
from sklearn.metrics import hamming_loss 
from transformers import (
    AutoTokenizer,
    TFAutoModelForSequenceClassification,
)

2022-12-21 12:52:03.555814: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tf.config.list_physical_devices()

2022-12-21 12:52:05.235303: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

2022-12-21 12:52:05.238101: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-21 12:52:05.238392: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
with zipfile.ZipFile('data/jigsaw-toxic-comment-classification-challenge.zip') as zipf:
    print(zipf.namelist())

['sample_submission.csv.zip', 'test.csv.zip', 'test_labels.csv.zip', 'train.csv.zip']


In [4]:
with zipfile.ZipFile('data/jigsaw-toxic-comment-classification-challenge.zip') as datazip:
    with zipfile.ZipFile(datazip.open('test.csv.zip')) as testzip:
        with testzip.open('test.csv') as testcsv:
            testdf = pd.read_csv(testcsv)
    with zipfile.ZipFile(datazip.open('test_labels.csv.zip')) as labelzip:
        with labelzip.open('test_labels.csv') as labelcsv:
            labeldf = pd.read_csv(labelcsv)

In [5]:
testdf.set_index('id', inplace=True)
labeldf.set_index('id', inplace=True)
testdf = testdf.join(labeldf)

In [6]:
testdf.head(3)

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,,
00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,-1,-1,-1,-1,-1,-1
0000247867823ef7,== From RfC == \n\n The title is fine as it is...,-1,-1,-1,-1,-1,-1
00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",-1,-1,-1,-1,-1,-1


In [7]:
scoredf = testdf[testdf.toxic >= 0]

tedf = pd.DataFrame(index=scoredf.index, data={
    'text': scoredf.comment_text,
    'label': scoredf.drop(columns='comment_text').values.tolist(),
})

In [8]:
testds = Dataset.from_pandas(tedf, preserve_index=False)

In [9]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_dataset(data):
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer(data['text'], padding='max_length', truncation=True)

testds = testds.map(tokenize_dataset)

100%|██████████| 63978/63978 [00:22<00:00, 2817.21ex/s]


In [10]:
model = TFAutoModelForSequenceClassification.from_pretrained(
    './toxicity-4batch-3epoch-unweighted',
)

2022-12-21 12:52:30.081547: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-21 12:52:30.084037: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-21 12:52:30.084532: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-21 12:52:30.084749: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_nod

In [11]:
tf_testds = model.prepare_tf_dataset(testds, batch_size=32, tokenizer=tokenizer)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [12]:
scoredf.head(32)

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,,
0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0
000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0
0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0
0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0,0,0,0,0,0
00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0,0,0,0,0,0
000663aff0fffc80,this other one from 1897,0,0,0,0,0,0
000689dd34e20979,== Reason for banning throwing == \n\n This ar...,0,0,0,0,0,0
000844b52dee5f3f,|blocked]] from editing Wikipedia. |,0,0,0,0,0,0
00091c35fa9d0465,"== Arabs are committing genocide in Iraq, but ...",1,0,0,0,0,0


In [13]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    http://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [14]:
labels = []
predictions = []
for inputs, targets in tf_testds:
    results = model.predict(inputs)
    labels.append(targets)
    predictions.append(tf.math.round(tf.nn.sigmoid(results.logits)))

y_true = tf.concat(labels, 0)
y_pred = tf.concat(predictions, 0)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
1/1 [==============================] - 1s 635ms/step


In [15]:
print(f'{hamming_score(y_true.numpy(), y_pred.numpy()) = }')
print(f'{hamming_loss(y_true.numpy(), y_pred.numpy()) = }')

hamming_score(y_true.numpy(), y_pred.numpy()) = 0.8985581332332835
hamming_loss(y_true.numpy(), y_pred.numpy()) = 0.030025429381357344


In [16]:
weighted_model = TFAutoModelForSequenceClassification.from_pretrained(
    './toxicity-4batch-3epoch',
)

Some layers from the model checkpoint at ./toxicity-4batch-3epoch were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at ./toxicity-4batch-3epoch and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
labels = []
predictions = []
for inputs, targets in tf_testds:
    results = weighted_model.predict(inputs)
    labels.append(targets)
    predictions.append(tf.math.round(tf.nn.sigmoid(results.logits)))

y_true = tf.concat(labels, 0)
y_pred = tf.concat(predictions, 0)

1/1 [==============================] - 0s 416ms/step


In [18]:
print(f'{hamming_score(y_true.numpy(), y_pred.numpy()) = }')
print(f'{hamming_loss(y_true.numpy(), y_pred.numpy()) = }')

hamming_score(y_true.numpy(), y_pred.numpy()) = 0.8268050692012672
hamming_loss(y_true.numpy(), y_pred.numpy()) = 0.15885807487076872
